**Eugen Ehrlich im Web of Science und in OpenAlex**

This notebook ...

# Preparation

In [33]:
# Uncomment and run once
# if you use vanilla python:
#!pip install py2neo python-dotenv pandas
# OR if you use anaconda:
#!conda install -c conda-forge py2neo python-dotenv
#!conda install panda
# for table of contents support, see
# https://github.com/ipython-contrib/jupyter_contrib_nbextensions

Import the required libraries and setup needed objects:

In [34]:
# config
from dotenv import dotenv_values
config = {**dotenv_values()}

# https://py2neo.org/v4/
from py2neo import Graph, Node, Relationship
graph = Graph(config['NEO4J_URL'])

# visualization
from scripts.vis import draw, table

# clean up visualization HTML dir
from pathlib import Path
import shutil
figure_dir = Path('figure')
if figure_dir.exists() and figure_dir.is_dir():
    shutil.rmtree(figure_dir)
figure_dir.mkdir()

# Web of Science

KB -> Neo4J

## Overview

In [35]:
cypher = """
use `ehrlich-wos`
match (a1:Author)-[r1:CREATOR_OF]->(w1:Work)<-[r2:CITES]-(w2:Work)<-[r3:CREATOR_OF]-(a2:Author)
where a1.display_name = "ehrlich, eugen"
return count(distinct w1) as works_cited,
    count(distinct r2) as num_citations,
    count(distinct w2) as citing_works,
    count(distinct a2) as citing_authors
"""
graph.run(cypher).data()

[{'works_cited': 33,
  'num_citations': 34,
  'citing_works': 25,
  'citing_authors': 37}]

## Cited works

In [4]:
cypher = """
use `ehrlich-wos`
match (a1:Author)-[r1:CREATOR_OF]->(w1:Work)<-[r2:CITES]-(w2:Work)<-[r3:CREATOR_OF]-(a2:Author)
where a1.display_name = "ehrlich, eugen"
return distinct a1.display_name as author, w1.year as year, w1.title as title order by w1.year
"""
graph.run(cypher).to_data_frame()

,author,year,title
0,"ehrlich, eugen",1893,no_title
1,"ehrlich, eugen",1906,die freie rechtsfindung
2,"ehrlich, eugen",1913,no_title
3,"ehrlich, eugen",1914,professor ehrlich's seminary of living law
4,"ehrlich, eugen",1916,montesquieu and sociological jurisprudence
5,"ehrlich, eugen",1916,das recht und die gesellschaft
6,"ehrlich, eugen",1917,judicial freedom of decision: its principles a...
7,"ehrlich, eugen",1921,judicial freedom of decision: principles and o...
8,"ehrlich, eugen",1922,the sociology of law
9,"ehrlich, eugen",1929,no_title


## Citing works

In [5]:
cypher = """
use `ehrlich-wos`
MATCH (a1:Author)-[:CREATOR_OF]->(w1:Work)<-[:CITES]-(w2:Work)<-[:CREATOR_OF]-(a2:Author)
WHERE a1.display_name = "ehrlich, eugen"
WITH w2, COLLECT(a2)[0] AS first_author
RETURN first_author.display_name AS author, w2.year AS year, w2.title AS title
ORDER BY w2.year
"""
graph.run(cypher).to_data_frame()

,author,year,title
0,"tamanaha, bz",2011,a vision of social-legal change: rescuing ehrl...
1,"krieger, linda h.",2011,when organizations rule: judicial deference to...
2,"wilder, colin f.",2012,teaching old dogs new tricks: four motifs of l...
3,"trajkovic, marko s.",2012,the realistic concept of the law
4,"kuenzler, adrian",2012,judicial legitimacy and the role of courts: ex...
5,"tamanaha, brian z.",2013,law's history: american legal thought and the ...
6,"martin, kimberly h.",2013,"social organization, collective sentiment, and..."
7,"exner, gudrun",2013,"the ""soziologische gesellschaft in wien"" [soci..."
8,"maurer, andreas",2013,the concept of participation in the making of ...
9,"zumbansen, peer",2013,law and legal pluralism: hybridity in transnat...


# OpenAlex

openalex.org -> neo4j

## Overview

In [6]:
cypher = """
use `ehrlich-openalex`
match (a1:Author)-[r1:CREATOR_OF]->(w1:Work)<-[r2:CITES]-(w2:Work)<-[r3:CREATOR_OF]-(a2:Author)
where a1.display_name = "ehrlich, eugen"
return count(distinct w1) as works_cited,
    count(distinct r2) as num_citations,
    count(distinct w2) as citing_works,
    count(distinct a2) as citing_authors
"""
graph.run(cypher).data()

[{'works_cited': 18,
  'num_citations': 402,
  'citing_works': 355,
  'citing_authors': 394}]

## Cited works (all years)

In [7]:
cypher = """
use `ehrlich-openalex`
match (a1:Author)-[r1:CREATOR_OF]->(w1:Work)<-[r2:CITES]-(w2:Work)<-[r3:CREATOR_OF]-(a2:Author)
match (w1)-[r4:PUBLISHED_IN]->(v1:Venue)
where a1.display_name = "ehrlich, eugen"
return distinct
    a1.display_name as author,
    w1.year as year,
    w1.title as title,
    v1.name as container,
    count(r2) as citations
order by citations desc
"""
graph.run(cypher).to_data_frame()

,author,year,title,container,citations
0,"ehrlich, eugen",1989,grundlegung der soziologie des rechts.,american journal of international law,145
1,"ehrlich, eugen",1938,fundamental principles of the sociology of law,university of chicago law review,138
2,"ehrlich, eugen",1922,the sociology of law,harvard law review,118
3,"ehrlich, eugen",1937,fundamental principles of the sociology of law,routledge ebooks,45
4,"ehrlich, eugen",1916,montesquieu and sociological jurisprudence,harvard law review,39
5,"ehrlich, eugen",1986,gesetz und lebendes recht,duncker & humblot ebooks,13
6,"ehrlich, eugen",1967,recht und leben,duncker & humblot ebooks,10
7,"ehrlich, eugen",1992,hans kelsen und die rechtssoziologie : auseina...,scientia verlag ebooks,5
8,"ehrlich, eugen",1913,soziologie des rechts,veit ebooks,4
9,"ehrlich, eugen",2003,rechtssoziologie und rechtswissenschaft : eine...,nomos ebooks,3


## Cited works (1960-1989)

In [8]:
cypher = """
use `ehrlich-openalex`
match (a1:Author)-[r1:CREATOR_OF]->(w1:Work)<-[r2:CITES]-(w2:Work)<-[r3:CREATOR_OF]-(a2:Author)
where a1.display_name = "ehrlich, eugen" and w2.year >= 1960 and w2.year <= 1990
return distinct
    a1.display_name as author,
    w1.year as year,
    w1.title as title,
    count(r2) as citations
order by citations desc
"""
graph.run(cypher).to_data_frame()

,author,year,title,citations
0,"ehrlich, eugen",1922,the sociology of law,20
1,"ehrlich, eugen",1938,fundamental principles of the sociology of law,16
2,"ehrlich, eugen",1989,grundlegung der soziologie des rechts.,6


## Citing works (all years)


In [9]:
cypher = """
use `ehrlich-openalex`
MATCH (a1:Author)-[:CREATOR_OF]->(w1:Work)<-[:CITES]-(w2:Work)<-[:CREATOR_OF]-(a2:Author)
WHERE a1.display_name = "ehrlich, eugen"
WITH w1, w2, COLLECT(a2)[0] AS first_author
RETURN
    first_author.display_name AS author,
    w2.year AS year,
    w2.title AS title,
    w1.title as cited_work
ORDER BY w2.year
"""
graph.run(cypher).to_data_frame()

,author,year,title,cited_work
0,"mannheim, hermann",1925,"die zwecke der rechtsmittel, insbesondere der ...",grundlegung der soziologie des rechts.
1,"otaka, tomoo",1932,die innere struktur des sozialen verbandes,grundlegung der soziologie des rechts.
2,"krader, lawrence",1960,the transition from serf to peasant in eastern...,fundamental principles of the sociology of law
3,"falk, richard a.",1961,"the relations of law to culture, power, and ju...",fundamental principles of the sociology of law
4,"chiba, m.",1962,right and duty,grundlegung der soziologie des rechts.
...,...,...,...,...
397,"geslin, albane",2022,dépasser les frontières disciplinaires dans la...,fundamental principles of the sociology of law
398,"rodi, michael",2022,theory of state decision-making,grundlegung der soziologie des rechts.
399,"scheiwe, kirsten",2022,"familie, gesellschaft und familienrecht",grundlegung der soziologie des rechts.
400,"botero-bernal, andrés",2022,el derecho social y la comunicación de ideas j...,fundamental principles of the sociology of law


## Citing works (1960-1989)

In [10]:
cypher = """
use `ehrlich-openalex`
MATCH (a1:Author)-[:CREATOR_OF]->(w1:Work)<-[:CITES]-(w2:Work)<-[:CREATOR_OF]-(a2:Author)
WHERE a1.display_name = "ehrlich, eugen" and w2.year > 1967 and w2.year < 1990
WITH w1, w2, COLLECT(a2)[0] AS first_author
RETURN
    first_author.display_name AS author,
    w2.year AS year,
    w2.title AS title,
    w1.title as cited_work
ORDER BY w2.year
"""
graph.run(cypher).to_data_frame()

,author,year,title,cited_work
0,"kulcsár, kálmán",1968,studies in the sociology of law in hungary,the sociology of law
1,"hahm, pyong choon",1969,religion and law in korea,fundamental principles of the sociology of law
2,"koch, klaus-friedrich",1969,law and anthropology: notes on interdisciplina...,fundamental principles of the sociology of law
3,"rueschemeyer, dietrich",1970,sociology of law in germany,grundlegung der soziologie des rechts.
4,"silva, edward f.",1970,states and the nation: state cohorts and amend...,the sociology of law
5,"ritterband, paul",1970,"law, policy, and behavior: educational exchang...",fundamental principles of the sociology of law
6,"gibbons, don l.",1971,observations on the study of crime causation,the sociology of law
7,"osborne, harry",1972,the use of sociological data in the judicial p...,fundamental principles of the sociology of law
8,"bonn, robert l.",1972,arbitration: an alternative system for handlin...,fundamental principles of the sociology of law
9,"pepinsky, harold b.",1973,the people v. the principle of legality in the...,the sociology of law


## Citing journals, all years

In [11]:
query = """
use `ehrlich-openalex`
MATCH (a1:Author)-[r1:CREATOR_OF]->(w1:Work)<-[r2:CITES]-(w2:Work)-[r3:PUBLISHED_IN]-(v1:Venue)
WHERE a1.display_name = "ehrlich, eugen"
with w1, v1, count(r2) as num_citations
where num_citations > 1
RETURN  v1.name AS container,
        num_citations,
        w1.title as cited_work
order by num_citations desc
"""
graph.run(query).to_data_frame()

,container,num_citations,cited_work
0,cambridge university press ebooks,22,grundlegung der soziologie des rechts.
1,cambridge university press ebooks,20,montesquieu and sociological jurisprudence
2,social science research network,15,fundamental principles of the sociology of law
3,springer ebooks,14,grundlegung der soziologie des rechts.
4,law & society review,6,the sociology of law
5,springer ebooks,5,gesetz und lebendes recht
6,law & society review,5,fundamental principles of the sociology of law
7,social science research network,4,fundamental principles of the sociology of law
8,springer ebooks,3,recht und leben
9,zeitschrift für rechtssoziologie,3,grundlegung der soziologie des rechts.


## Graph: Citing journal, all years

In [36]:
query = """
use `ehrlich-openalex`
MATCH (a1:Author)-[r1:CREATOR_OF]->(w1:Work)<-[r2:CITES]-(w2:Work)-[r3:PUBLISHED_IN]-(v1:Venue)
WHERE a1.display_name = "ehrlich, eugen"
with w1, v1, count(r2) as num_citations
where num_citations > 1
RETURN  v1 AS source_node,
        apoc.create.vRelationship(v1, 'CITATIONS', {value:num_citations}, w1) as rel,
        w1 as target_node
"""
options = {"Author": "display_name",
           "Work": "display_name",
           "Venue": "name",
           "CITATIONS":"value"}
draw(graph, query, options, physics=True)

## Citing journals, 1960-1989

In [13]:
query = """
use `ehrlich-openalex`
MATCH (a1:Author)-[r1:CREATOR_OF]->(w1:Work)<-[r2:CITES]-(w2:Work)-[r3:PUBLISHED_IN]-(v1:Venue)
WHERE a1.display_name = "ehrlich, eugen" and w2.year > 1960 and w2.year < 1990
with w1, v1, count(r2) as num_citations
where num_citations > 1
RETURN  v1 AS source_node,
        apoc.create.vRelationship(v1, 'CITATIONS', {value:num_citations}, w1) as rel,
        w1 as target_node
"""
options = {"Author": "display_name", "Work": "display_name", "Venue": "name", "CITATIONS":"value"}
draw(graph, query, options, physics=True)

## Citing authors, 1960-1989

In [10]:
query = """
MATCH (a1:Author)-[r1:CREATOR_OF]->(w1:Work)<-[r2:CITES]-(w2:Work)<-[r3:CREATOR_OF]-(a2:Author)
WHERE a1.display_name = "ehrlich, eugen" and w2.year > 1960 and w2.year < 1990
with a1, a2, count(r2) as num_citations
where num_citations > 1
RETURN  a2 AS source_node,
        apoc.create.vRelationship(a2, 'CITES', {value:num_citations}, a1) as rel,
        a1 as target_node
"""
options = {"Author": "display_name", "Work": "display_name", "Venue": "name", "CITES":"value"}
draw(graph, query, options, physics=True, db="ehrlich-openalex")

# Eigene Daten

## Citing authors, 1960-1989

In [5]:
query = """
MATCH (a1:Author)-[r1:CREATOR_OF]->(w1:Work)<-[r2:CITES]-(w2:Work)<-[r3:CREATOR_OF]-(a2:Author)
WHERE toLower(a1.family) = "ehrlich" //and w2.year > 1960 and w2.year < 1990
with a1, a2, count(r2) as num_citations
where num_citations > 0
RETURN  a2 AS source_node,
        apoc.create.vRelationship(a2, 'CITES', {value:num_citations}, a1) as rel,
        a1 as target_node
"""
options = {"Author": "family", "Work": "display_name", "Venue": "name", "CITES":"value"}
draw(graph, query, options, physics=True, db="zfrsoz")

## Citing works, all years

In [25]:
query = """
MATCH (a1:Author)-[:CREATOR_OF]->(w1:Work)<-[:CITES]-(w2:Work)<-[:CREATOR_OF]-(a2:Author)
WHERE a1.family = "ehrlich" and a1.given starts with "e"
WITH w2, COLLECT(a2)[0] AS first_author, COLLECT(w1)[0].title AS most_cited, count(w2) as count
RETURN
    first_author.display_name AS author,
    w2.year AS year,
    w2.title AS title,
    most_cited,
    count
ORDER BY count DESC, year
"""


### Zeitschrift für Rechtssoziologie

In [26]:
table(graph, query, db="zfrsoz")

,author,year,title,most_cited,count
0,"Haferkamp, Hans-Peter",2021,hermann kantorowicz’ vortrag auf dem ersten so...,freie rechtsfindung und freie rechtswissenschaft,5
1,"Fuchs, Gesine",2016,zu häufigkeit und determinanten rechtlicher be...,rechtsfaehigkeit,2
2,"Kretschmann, Andrea",2019,recht in bewegung,grundlegung der soziologie des rechts,2
3,"Boehncke, Clemens",2021,die ‚freirechtler‘ als intellektuellengruppe b...,die tatsachen des gewohnheitsrechts,2
4,"Schrör, Simon",2022,"zwischen wissenschaft, praxis und politik<b>: ...",grundlegung der soziologie des rechts,2
5,"Kargl, Waller",1991,gesellschaft ohne subjekte oder subjekte ohne ...,grundlegung der soziologie des rechts,1
6,"Noelle-Neumann, Elisabeth",1995,rechtsbewußtsein im wiedervereinigten deutschland,das lebende recht der völker der bukowina (191...,1
7,"Fischer-Lescano, Andreas",2002,globalverfassung: los desaparecidos und das pa...,grundlegung der soziologie des rechts,1
8,"Schulz-Schaeffer, Ingo",2004,rechtsdogmatik als gegenstand der rechtssoziol...,über lücken im rechte,1
9,"Rottleuthner, Hubert",2013,das lebende recht bei eugen ehrlich und ernst ...,recht und leben,1


### Lehrbücher

In [27]:
table(graph, query, db="rsoz-lehrbuecher")

,author,year,title,most_cited,count
0,"raiser, thomas",1987,rechtssoziologie,entgegnung ebd,7
1,"rehbinder, manfred",1989,rechtssoziologie,die neuordnung der gerichtsverfassung,6
2,"girtler, r.",1976,rechtssoziologie,recht und leben. gesammelte schriften zur rech...,5
3,"rottleuthner, hubert",1987,einführung in die rechtssoziologie,recht und leben,5
4,"rehbinder, manfred",1977,rechtssoziologie,recht und leben,4
5,"ryffel, hans",1974,rechtssoziologie. eine systematische orientierung,soziologie der vorurteile,3
6,"raiser, thomas",1973,einführung in die rechtssoziologie,recht und leben. gesammelte schriften zur rech...,2
7,"dux, günter",1978,rechtssoziologie eine einführung,grundlegung der soziologie des rechts (1913,2
8,"luhmann, niklas",1983,rechtssoziologie,grundlegung der soziologie des rechts (1913,2
9,"geiger, theodor-",1964,vorstudien zu einer soziologie des rechts,"grundlegung der soziologie der rechts, unverän...",1


### JLS

In [28]:
table(graph, query, db="jls")

,author,year,title,most_cited,count
0,"hertogh, marc",2004,a 'european' conception of legal consciousness...,`das lebende recht der voèlker der bukowina',7
1,"coutu, michel",2011,john r. commons and max weber: the foundations...,grundlegung der soziologie des rechts,5
2,"economides, kim",1986,the spatial analysis of legal systems: towards...,fundamental principles of the sociology of law,3
3,"stewart, iain",1990,the critical legal science of hans kelsen,'judicial freedom of decision: its principles ...,2
4,"priban, j",2012,constitutionalism as fear of the political? a ...,fundamental principles of the sociology of law,2
5,"coutu, michel",2013,weber reading stammler: what horizons for the ...,"grundlegung der soziologie des rechts, (1913/1...",2
6,"přibáň, jiří",2017,a sociology of legal distinctions: introducing...,fundamental principles of the sociology of law,2
7,"coutu, michel",2020,"economic crises, crisis of labour law? lessons...",grundlegung der soziologie des rechts,2
8,"carlen, pat",1974,remedial routines for the maintenance of contr...,the sociologyof law,1
9,"king, michael",1991,child welfare within law: the emergence of a h...,fundamental principles of the sociology of law,1


## Citing works, 1970-1989

In [29]:
query = """
MATCH (a1:Author)-[:CREATOR_OF]->(w1:Work)<-[:CITES]-(w2:Work)<-[:CREATOR_OF]-(a2:Author)
WHERE a1.family = "ehrlich" and a1.given starts with "e" 
AND w2.year > 1960 and w2.year < 1990 
WITH w2, COLLECT(a2)[0] AS first_author, COLLECT(w1)[0].title AS most_cited, count(w2) as count
RETURN
    first_author.display_name AS author,
    w2.year AS year,
    w2.title AS title,
    most_cited,
    count
ORDER BY count DESC, year
"""

### Zeitschrift für Rechtssoziologie

In [30]:
table(graph, query, db="zfrsoz")

""


### Lehrbücher

In [31]:
table(graph, query, db="rsoz-lehrbuecher")

,author,year,title,most_cited,count
0,"raiser, thomas",1987,rechtssoziologie,entgegnung ebd,7
1,"rehbinder, manfred",1989,rechtssoziologie,die neuordnung der gerichtsverfassung,6
2,"girtler, r.",1976,rechtssoziologie,recht und leben. gesammelte schriften zur rech...,5
3,"rottleuthner, hubert",1987,einführung in die rechtssoziologie,recht und leben,5
4,"rehbinder, manfred",1977,rechtssoziologie,recht und leben,4
5,"ryffel, hans",1974,rechtssoziologie. eine systematische orientierung,soziologie der vorurteile,3
6,"raiser, thomas",1973,einführung in die rechtssoziologie,recht und leben. gesammelte schriften zur rech...,2
7,"dux, günter",1978,rechtssoziologie eine einführung,grundlegung der soziologie des rechts (1913,2
8,"luhmann, niklas",1983,rechtssoziologie,grundlegung der soziologie des rechts (1913,2
9,"geiger, theodor-",1964,vorstudien zu einer soziologie des rechts,"grundlegung der soziologie der rechts, unverän...",1


### JLS

In [32]:
table(graph, query, db="jls")

,author,year,title,most_cited,count
0,"economides, kim",1986,the spatial analysis of legal systems: towards...,fundamental principles of the sociology of law,3
1,"carlen, pat",1974,remedial routines for the maintenance of contr...,the sociologyof law,1
